# RESOW download composite S2 image

This code has been written by Clive Neil (2022) for the NOC project ReSow with th eaim of using remote sensed data to delineate the presence of seagrass around the UK coast.

Retrieval of median composite Sentinel-2 images of the case study regions from Google Earth Engine.


## Initial settings

Refer to the Set-up and Installation section of the User Handbook for instructions on how to install the Python packages necessary to run the software, including Google Earth Engine Python API. See original methodology via https://github.com/kvos/CoastSat

In [ ]:
import os
import sys
import json
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from utils.print_utils import printProgress, printSuccess, printWarning

from gee_connection import downloader, preprocess,  tools

from global_parameters import BAND_DICT, SWIR


# directory where the data will be accessed and stored
data_partition = 'd:\\data\\resow'
sites_dir_path = os.path.join(data_partition, 'sites')

sites = os.listdir(sites_dir_path)

summer = [
         ['2021-06-01', '2021-08-31']
         ]

dates = summer

roi = [-4.57, 52.94] # Portdinllaen
roi = [-2.95, 59.28] # Tuquoy

download = True

## download optical data from GEE

In [ ]:
if download:
    for site in sites:

        kml_filepath = os.path.join(sites_dir_path, site)    
        kml_polygon = tools.polygon_from_kml(kml_filepath)
        roi_polygon = tools.smallest_rectangle(kml_polygon)

        site_name = site[:site.find('.')]
        median_dir_path = os.path.join(data_partition, site_name, 'median')

        for date_pair in dates:

            printProgress(f'processing {site_name}: {date_pair}')
            printProgress('')

            downloader.getS2GEEImage(site_name, roi_polygon, date_pair, median_dir_path)    

        downloader.save_metadata(site_name, median_dir_path)

In [ ]:
if False:
    for NIR_index, NIR_band in enumerate(BAND_DICT['10m'][0]):
        if NIR_band == 'B8':
            break
            
    for site in sites:
        site_name = site[:site.find('.')]
        median_dir_path = os.path.join(data_partition, site_name, 'median')

        for date_pair in dates:
            metadata = downloader.load_metadata(site_name, median_dir_path, date_pair)
            file_names = metadata['file_names']

            file_paths = []
            for band_index, band_key in enumerate(BAND_DICT.keys()):
                file_paths.append(os.path.join(median_dir_path, band_key, file_names[band_index]))

            images_ms, geometry = preprocess.preprocess_optical(file_paths,
                                      pansharpen=True,
                                      NIR_index=NIR_index)
            images_ms = np.moveaxis(images_ms, 2, 0)

            print(images_ms.shape)

In [ ]:
file_name = os.path.join(median_dir_path, site_name + '_median_6bands_add.tif')
preprocess.writeGeotiff(images_ms, file_name, geometry)

In [ ]:
file_name